# Tarea 2: Procesamiento de Lenguaje Natural 
## Sergio Alberto De León Martínez 

In [1]:
Nombre = 'Sergio Alberto De León Martínez'
Num_tarea = 2

In [17]:
import nltk
import numpy as np

## 2 Bolsas de Palabras, Bigramas y Emociones (40pts)

1. Evalué BoW con pesado binario.

In [34]:
# Obtenemos los datos 
def get_texts_from_file(path_corpus, path_truth):
    tr_text = []
    tr_y = []

    with open(path_corpus, 'r') as f_corpus, open(path_truth, 'r') as f_truth:
        for twitt in f_corpus:
            tr_text += [twitt]
        for label in f_truth:
            tr_y += [label]
    return tr_text, tr_y
    
tr_text, tr_y = get_texts_from_file('./mex20_train.txt', './mex20_train_labels.txt')
val_txt, val_y = get_texts_from_file('./mex20_val.txt', './mex20_val_labels.txt')


In [7]:
# Tokenizamos el corpus 
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()
corpus = []
for doc in tr_text:
    corpus += tokenizer.tokenize(doc)

fdist = nltk.FreqDist(corpus)
print('Longitud del corpus:', len(corpus))
fdist

Longitud del corpus: 97473


FreqDist({',': 3016, 'de': 2915, 'que': 2829, '.': 2604, 'la': 2031, 'a': 1956, 'y': 1856, '!': 1435, 'no': 1430, '@USUARIO': 1399, ...})

In [18]:
# Ordenamos el corpus 
corpus_ordenado = [(fdist[key], key) for key in fdist]
corpus_ordenado.sort()
corpus_ordenado.reverse()

corpus_filtrado = corpus_ordenado[:5000]
corpus_filtrado_dict = {}
for frecuencia, token in corpus_filtrado:
    corpus_filtrado_dict[token] = frecuencia
print('Corpus con los 5000 tokens más frecuentes:', corpus_filtrado_dict)

# Creamos el ranking 
corpus_ranking = {}
cont = 0
for frecuencia, palabra in corpus_filtrado:
    corpus_ranking[palabra] = cont
    cont += 1
print('Ranking de tokens en el corpus:', corpus_ranking)

Corpus con los 5000 tokens más frecuentes: {',': 3016, 'de': 2915, 'que': 2829, '.': 2604, 'la': 2031, 'a': 1956, 'y': 1856, '!': 1435, 'no': 1430, '@USUARIO': 1399, 'me': 1198, 'el': 1043, 'en': 1018, 'se': 851, 'es': 736, 'con': 706, '?': 704, 'verga': 691, 'los': 683, 'madre': 665, 'por': 612, 'las': 601, '"': 590, 'un': 582, 'te': 550, 'mi': 547, 'lo': 509, 'putas': 493, 'una': 472, '...': 464, 'putos': 462, 'para': 458, '😂': 434, 'si': 422, 'ya': 420, 'como': 416, 'su': 403, 'pero': 377, 'tu': 365, 'loca': 357, 'le': 350, 'más': 325, 'No': 318, 'del': 298, 'gorda': 294, 'al': 284, 'bien': 252, 'A': 249, '¿': 243, 'Y': 242, 'son': 229, 'Me': 229, 'o': 227, 'feas': 225, 'cuando': 225, 'Que': 214, ':': 208, 'yo': 205, 'les': 199, 'porque': 196, 'ni': 191, 'está': 191, 'ser': 189, 'estoy': 184, 'sus': 181, 'todos': 172, 'esta': 169, 'puta': 168, 'Ya': 167, 'todo': 166, 'pinche': 163, 'puto': 161, 'tan': 160, 'Si': 157, 'La': 157, 'qué': 154, '…': 151, 'eso': 149, 'muy': 147, 'soy': 14

In [99]:
# Creamos una clase para constuir la bolsa de palabras 
from collections import Counter

class BOW:
    def __init__(self, tr_text, corpus_ranking):
        self.tr_text = tr_text
        self.corpus_ranking = corpus_ranking 
        self._N = len(tr_text)
        self._M = len(corpus_ranking)
        self.bow = np.zeros((self._N, self._M))

    def _doc_freq(self):
        '''
            Función que cuenta el número de veces que aparecen los 
            terminos del vocabulario en los documentos 
        '''
        doc_freq_dict = {}
        tweets_tokenizados = [Counter(tokenizer.tokenize(doc)) for doc in tr_text]
        for token in corpus_ranking:
            if token in doc_freq_dict:
                pass
            else:
                doc_freq_dict[token] = sum([1 for tweet_tokenizado in tweets_tokenizados if token in tweet_tokenizado])
        return doc_freq_dict

    def bow_pesado(self, pesado='binario'):
        '''
            Función para construir la bolsa de palabras 
        '''
        if pesado == 'tf_idf':
            doc_freq_dict = self._doc_freq()
            
        for num_doc, doc in enumerate(self.tr_text):
            doc_tokenizado = Counter(tokenizer.tokenize(doc))
            for palabra, count in doc_tokenizado.items():
                if palabra in self.corpus_ranking:
                    idx = self.corpus_ranking[palabra]
                    if pesado == 'binario':
                        self.bow[num_doc, idx] = 1
                    elif pesado == 'frecuencia':
                        self.bow[num_doc, idx] = count
                    elif pesado == 'tf_idf':
                        self.bow[num_doc, idx] = count * np.log(self._N / doc_freq_dict[palabra])
        return self.bow

    def bow_normalizado(self):
        for i in range(self._M):
            norm = np.linalg.norm(self.bow[:, i])
            if norm > 0:
                self.bow[:, i] /= norm
        return self.bow


In [37]:
# Pesado binario 
bow_tb = BOW(tr_text, corpus_ranking)
bow_train_binario = bow_tb.bow_pesado(pesado='binario')

bow_vb = BOW(val_txt, corpus_ranking)
bow_val_binario = bow_vb.bow_pesado(pesado='binario')

In [39]:
# Clasificación 
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_fscore_support

tr_y = list(map(int, tr_y))
val_y = list(map(int, val_y))

def clasificacion(bow_train, bow_val, tr_y, val_y): 
    '''
        Función para clasificar con SVM
    '''
    parameters = {'C' : [.05, .12, .25, .5, 1, 2, 4]}

    svr = svm.LinearSVC(class_weight='balanced')
    grid = GridSearchCV(
        estimator=svr, 
        param_grid=parameters, 
        n_jobs=8, 
        scoring='f1_macro', 
        cv=5
    )

    grid.fit(bow_train, tr_y)
    y_pred = grid.predict(bow_val)

    p, r, f, _ = precision_recall_fscore_support(
        val_y, 
        y_pred, 
        average='macro', 
        pos_label=1
    )

    print(confusion_matrix(val_y, y_pred))
    print(metrics.classification_report(val_y, y_pred))

clasificacion(bow_train_binario, bow_val_binario, tr_y, val_y)

/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[[356  62]
 [ 49 120]]
              precision    recall  f1-score   support

           0       0.88      0.85      0.87       418
           1       0.66      0.71      0.68       169

    accuracy                           0.81       587
   macro avg       0.77      0.78      0.77       587
weighted avg       0.82      0.81      0.81       587



2. Evalué BoW con pesado frecuencia.

In [40]:
# Pesado frecuencia 
bow_tf = BOW(tr_text, corpus_ranking)
bow_train_frecuencia = bow_tf.bow_pesado(pesado='frecuencia')

bow_vf = BOW(val_txt, corpus_ranking)
bow_val_frecuencia = bow_vf.bow_pesado(pesado='frecuencia')

In [44]:
clasificacion(bow_train_frecuencia, bow_val_frecuencia, tr_y, val_y)

/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/a

[[362  56]
 [ 45 124]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       418
           1       0.69      0.73      0.71       169

    accuracy                           0.83       587
   macro avg       0.79      0.80      0.79       587
weighted avg       0.83      0.83      0.83       587



3. Evalué BoW con pesado tfidf.

In [90]:
# Pesado tfidf
bow_ttfidf = BOW(tr_text, corpus_ranking)
bow_train_tfidf = bow_ttfidf.bow_pesado(pesado='tf_idf')

bow_vtfidf = BOW(val_txt, corpus_ranking)
bow_val_tfidf = bow_vtfidf.bow_pesado(pesado='tf_idf')

In [91]:
clasificacion(bow_train_tfidf, bow_val_tfidf, tr_y, val_y)

/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/a

[[355  63]
 [ 94  75]]
              precision    recall  f1-score   support

           0       0.79      0.85      0.82       418
           1       0.54      0.44      0.49       169

    accuracy                           0.73       587
   macro avg       0.67      0.65      0.65       587
weighted avg       0.72      0.73      0.72       587



4. Evalué BoW con pesado binario normalizado l2 (no use sklearn).

In [110]:
bow_tb.bow_normalizado()
bow_train_binario_normalizado = bow_tb.bow

bow_vb.bow_normalizado()
bow_val_binario_normalizado = bow_vb.bow

In [115]:
clasificacion(
    bow_train_binario_normalizado, 
    bow_val_binario_normalizado, 
    tr_y, 
    val_y
)

[[339  79]
 [ 38 131]]
              precision    recall  f1-score   support

           0       0.90      0.81      0.85       418
           1       0.62      0.78      0.69       169

    accuracy                           0.80       587
   macro avg       0.76      0.79      0.77       587
weighted avg       0.82      0.80      0.81       587



5. Evalué BoW con pesado frecuencia normalizado l2 (no use sklearn).

In [116]:
bow_tf.bow_normalizado()
bow_train_frecuencia_normalizado = bow_tf.bow

bow_vf.bow_normalizado()
bow_val_frecuencia_normalizado = bow_vf.bow

In [117]:
clasificacion(
    bow_train_frecuencia_normalizado,
    bow_val_frecuencia_normalizado, 
    tr_y, 
    val_y
)

[[337  81]
 [ 35 134]]
              precision    recall  f1-score   support

           0       0.91      0.81      0.85       418
           1       0.62      0.79      0.70       169

    accuracy                           0.80       587
   macro avg       0.76      0.80      0.78       587
weighted avg       0.82      0.80      0.81       587



6. Evalué BoW con pesado tfidf normalizado l2 (no use sklearn).

In [118]:
bow_ttfidf.bow_normalizado()
bow_train_tfidf_normalizado = bow_ttfidf.bow

bow_vtfidf.bow_normalizado()
bow_val_tfidf_normalizado = bow_vtfidf.bow

In [119]:
clasificacion(
    bow_train_tfidf_normalizado, 
    bow_val_tfidf_normalizado, 
    tr_y, 
    val_y
)

[[344  74]
 [ 42 127]]
              precision    recall  f1-score   support

           0       0.89      0.82      0.86       418
           1       0.63      0.75      0.69       169

    accuracy                           0.80       587
   macro avg       0.76      0.79      0.77       587
weighted avg       0.82      0.80      0.81       587



7. Ponga una tabla comparativa a modo de resumen con las seis entradas anteriores.

Elegimos como métrica el valor del F1-score (weighted avg), ya que toma en cuenta otras métricas como precision y recall, además cosidera el hecho de que el tamaño de las clases esta desbalanceado. Como podemos ver, el mejor redimiento lo obtuvo el pesado frecuencia sin normalizar, en este caso la frecuencia de terminos se vuelve relevante porque los documentos, en este caso twits, son relativamente pequeños.

| Pesado | Normalizado | F1-score (weighted avg) |
|:------------:|:--------------:|:-------------:|
| Binario         | No         |         0.81 |
| Frecuencia      | No         |         0.83 |
| TF IDF          | No         |         0.72 |
| Binario         | Si         |         0.81 |
| Frecuencia      | Si         |         0.81 |
| TF IDF          | Si         |         0.81 |



8. De las configuraciones anteriores elija la mejor y evalúela con más y menos términos (e.g., 1000 y 7000). Ponga una tabla dónde compare las tres configuraciones.

In [356]:
# Usemos bolsa con pesado frecuencia 
# Para 1000 terminos 
corpus_filtrado_1000 = corpus_ordenado[:1000]
corpus_ranking_1000 = {}
for i, (_, token) in enumerate(corpus_filtrado_1000):
    corpus_ranking_1000[token] = i

bow_t1000 = BOW(tr_text=tr_text, corpus_ranking=corpus_ranking_1000)
bow_t1000.bow_pesado(pesado='frecuencia')
bow_train_1000 = bow_t1000.bow

bow_v1000 = BOW(tr_text=val_txt, corpus_ranking=corpus_ranking_1000)
bow_v1000.bow_pesado(pesado='frecuencia')
bow_val_1000 = bow_v1000.bow

clasificacion(bow_train_1000, bow_val_1000, tr_y, val_y)

/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/a

[[350  68]
 [ 43 126]]
              precision    recall  f1-score   support

           0       0.89      0.84      0.86       418
           1       0.65      0.75      0.69       169

    accuracy                           0.81       587
   macro avg       0.77      0.79      0.78       587
weighted avg       0.82      0.81      0.81       587



In [357]:
# Para 7000 terminos 
corpus_filtrado_7000 = corpus_ordenado[:7000]
corpus_ranking_7000 = {}
for i, (_, token) in enumerate(corpus_filtrado_7000):
    corpus_ranking_7000[token] = i

bow_t7000 = BOW(tr_text=tr_text, corpus_ranking=corpus_ranking_7000)
bow_t7000.bow_pesado(pesado='frecuencia')
bow_train_7000 = bow_t7000.bow

bow_v7000 = BOW(tr_text=val_txt, corpus_ranking=corpus_ranking_7000)
bow_v7000.bow_pesado(pesado='frecuencia')
bow_val_7000 = bow_v7000.bow

clasificacion(bow_train_7000, bow_val_7000, tr_y, val_y)

/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sergio/a

[[360  58]
 [ 52 117]]
              precision    recall  f1-score   support

           0       0.87      0.86      0.87       418
           1       0.67      0.69      0.68       169

    accuracy                           0.81       587
   macro avg       0.77      0.78      0.77       587
weighted avg       0.81      0.81      0.81       587



Para este experimento elegimos el pesado frecuencia sin normalizar y obtuvimos los siguientes resultado:

| No. De términos | F1-score (weighted avg) |
|:------------:|:--------------:|
| 1000      | 0.81         |         
| 5000      | 0.83         |        
| 7000      | 0.81         |         

Como podemos ver, usando un numero diferente de términos realmente no mejora la calidad del modelo 

9. Utilice el recurso léxico del Consejo Nacional de Investigación de Canadá llamado
"EmoLex" (https://www.saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm) para
construir una "Bolsa de Emociones" de los Tweets de agresividad (Debe usar EmoLex
en Español). Para esto, una estrategia sencilla sería enmascarar cada palabra con su
emoción, y después construir la Bolsa de Emociones (BoE).

Primero vamos a crear un diccionario que contenga la palabra junto con su emoción asociada codificada en un vector one-hot dónde la emociones estan ordenadas como: [anger, fear, anticipation, trust, surprise, sadness, joy, disgust]

In [369]:
emolex_dict = {} # palabra : emoción (índice)

with open('./Spanish-NRC-EmoLex.txt', 'r') as emolex_file:
    next(emolex_file)  # Salta la primera línea de encabezados
    for word_enriched in emolex_file:
        parts = word_enriched.split()
        one_hot_emotion = np.array(parts[1:6] + parts[8:11], dtype=float)
        word_spanish = ' '.join(parts[11:]).lower()
        if word_spanish in emolex_dict:
            pass
        else:
            emolex_dict[word_spanish] = one_hot_emotion


In [370]:
print(emolex_dict)

{'detrás': array([0., 0., 0., 0., 0., 0., 0., 0.]), 'ábaco': array([0., 0., 0., 0., 0., 0., 0., 1.]), 'abandonar': array([0., 0., 0., 1., 0., 1., 0., 0.]), 'abandonado': array([1., 0., 0., 1., 0., 1., 0., 0.]), 'abandono': array([1., 0., 0., 1., 0., 1., 1., 0.]), 'disminuir': array([0., 0., 0., 0., 0., 0., 0., 0.]), 'disminución': array([0., 0., 0., 0., 0., 0., 0., 0.]), 'abba': array([0., 0., 0., 0., 0., 0., 0., 0.]), 'abad': array([0., 0., 0., 0., 0., 0., 0., 1.]), 'abreviar': array([0., 0., 0., 0., 0., 0., 0., 0.]), 'abreviatura': array([0., 0., 0., 0., 0., 0., 0., 0.]), 'abdomen': array([0., 0., 0., 0., 0., 0., 0., 0.]), 'abdominal': array([0., 0., 0., 0., 0., 0., 0., 0.]), 'secuestro': array([0., 0., 0., 1., 0., 1., 1., 0.]), 'aberrante': array([0., 0., 0., 0., 0., 0., 0., 0.]), 'aberración': array([0., 0., 1., 0., 0., 0., 0., 0.]), 'suspensión': array([0., 0., 0., 0., 0., 0., 0., 0.]), 'aborrecer': array([1., 0., 1., 1., 0., 0., 0., 0.]), 'aborrecible': array([1., 0., 1., 1., 0.,

Ahora elaboremos la bolsa de emociones. Para constuir la representacion de cada documento elegí acumular la codificación one-hot de cada palabra, me parecio una estrategia útil ya que de esta forma tenemos registro de las emociones presentes y la frecuencia con la que aparecen 

In [373]:
# Creamos una clase para constuir la bolsa de emociones  
from collections import Counter

class BOW_emotions:
    def __init__(self, tr_text, corpus_ranking, emolex_dict):
        self.tr_text = tr_text
        self.corpus_ranking = corpus_ranking  
        self._N = len(tr_text)
        self._M = 8 # Número de emociones 
        self.bow = np.zeros((self._N, self._M))
        self.emolex_dict = emolex_dict

    def _doc_freq(self):
        """
            Función para contar el número de documentos 
            que contienen un termino 
        """
        doc_freq_dict = {}
        tweets_tokenizados = [Counter(tokenizer.tokenize(doc)) for doc in tr_text]
        for token in corpus_ranking:
            if token in doc_freq_dict:
                pass
            else:
                doc_freq_dict[token] = sum([1 for tweet_tokenizado in tweets_tokenizados if token in tweet_tokenizado])
        return doc_freq_dict

    def bow_pesado(self, pesado='binario'):
        """
            Función para rellenar la bolsa de palabras 
            de acuerdo al pesado solicitado 
        """
        if pesado == 'tf_idf':
            doc_freq_dict = self._doc_freq()
            
        for num_doc, doc in enumerate(self.tr_text):
            doc_tokenizado = Counter(tokenizer.tokenize(doc))
            for palabra, count in doc_tokenizado.items():
                if palabra in self.corpus_ranking and palabra in self.emolex_dict:
                    if pesado == 'binario':
                        self.bow[num_doc] = self.bow[num_doc] + self.emolex_dict[palabra]
                    elif pesado == 'frecuencia':
                        self.bow[num_doc] = self.bow[num_doc] + count * self.emolex_dict[palabra]
                    elif pesado == 'tf_idf':
                        self.bow[num_doc] = self.bow[num_doc] + count * np.log(self._N / doc_freq_dict[palabra]) * self.emolex_dict[palabra]
        return self.bow

    def bow_normalizado(self):
        for i in range(self._M):
            norm = np.linalg.norm(self.bow[:, i])
            if norm > 0:
                self.bow[:, i] /= norm
        return self.bow


10. Evalúa tú BoE clasificando con SVM. Ponga una tabla comparativa a modo de resumen con los tres pesados, normalize cada uno si lo cree conveniente.

In [374]:
# Pesado binario sin normalizar 
bow_etb = BOW_emotions(tr_text, corpus_ranking, emolex_dict)
bow_emotions_train_binario = bow_etb.bow

bow_evb = BOW_emotions(val_txt, corpus_ranking, emolex_dict)
bow_emotions_val_binario = bow_evb.bow

In [375]:
clasificacion(bow_emotions_train_binario, bow_emotions_val_binario, tr_y, val_y)

[[418   0]
 [169   0]]
              precision    recall  f1-score   support

           0       0.71      1.00      0.83       418
           1       0.00      0.00      0.00       169

    accuracy                           0.71       587
   macro avg       0.36      0.50      0.42       587
weighted avg       0.51      0.71      0.59       587



/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [376]:
# Pesado frecuencia normalizado 
bow_etf = BOW_emotions(tr_text, corpus_ranking, emolex_dict)
bow_etf.bow_pesado(pesado='frecuencia')
bow_etf.bow_normalizado()
bow_emotions_train_frecuencia = bow_etf.bow

bow_evf = BOW_emotions(val_txt, corpus_ranking, emolex_dict)
bow_evf.bow_pesado(pesado='frecuencia')
bow_evf.bow_normalizado()
bow_emotions_val_frecuencia = bow_evf.bow

In [377]:
clasificacion(bow_emotions_train_frecuencia, bow_emotions_val_frecuencia, tr_y, val_y)

[[272 146]
 [ 91  78]]
              precision    recall  f1-score   support

           0       0.75      0.65      0.70       418
           1       0.35      0.46      0.40       169

    accuracy                           0.60       587
   macro avg       0.55      0.56      0.55       587
weighted avg       0.63      0.60      0.61       587



In [378]:
# Pesado TF IDF normalizado
bow_ettfidf = BOW_emotions(tr_text, corpus_ranking, emolex_dict)
bow_ettfidf.bow_pesado(pesado='tf_idf')
bow_ettfidf.bow_normalizado()
bow_emotions_train_tfidf = bow_ettfidf.bow

bow_evtfidf = BOW_emotions(val_txt, corpus_ranking, emolex_dict)
bow_evtfidf.bow_pesado(pesado='tf_idf')
bow_evtfidf.bow_normalizado()
bow_emotions_val_tfidf = bow_evtfidf.bow 

In [379]:
clasificacion(bow_emotions_train_tfidf, bow_emotions_val_tfidf, tr_y, val_y)

[[377  41]
 [139  30]]
              precision    recall  f1-score   support

           0       0.73      0.90      0.81       418
           1       0.42      0.18      0.25       169

    accuracy                           0.69       587
   macro avg       0.58      0.54      0.53       587
weighted avg       0.64      0.69      0.65       587



10. Evalúa tú BoE clasificando con SVM. Ponga una tabla comparativa a modo de resumen
con los tres pesados, normalize cada uno si lo cree conveniente.

A continuación se muestran los resultados, en este caso el mejor rendimiento lo tiene el pesado TF IDF normalizado, cabe resaltar que para el pesado binario, de manera sistemática predice correctamente los elementos de una clase, pero no los de la otra.

| Método | F1-score (weighted avg) |
|:------------:|:--------------:|
| Pesado binario (sin normalizar) | 0.59 | 
| Pesado frecuencia (normalizado) | 0.61 |
| Pesado TF IDF (normalizado)     | 0.65 |

## 3 Recurso Línguistico de Emociones Mexicano (30 pts)

1. Utilice el recurso léxico llamado "Spanish Emotion Lexicon (SEL)" del Dr. Grigori Sidorov, profesor del Centro de Investigación en Computación (CIC) del Instituto Politécnico Nacional (http://www.cic.ipn.mx/∼sidorov/), para enmascarar cada palabra con su emoción, y después construir la Bolsa de Emociones con algún pesado (e.g., binario, tf, tfidf). Proponga alguna estrategia para incorporar el "valor" del "Probability Factor of Affective use" en su representación vectorial del documento. Evalúa y escribe una tabla comparativa a modo de resumen con al menos tres pesados: binario, frecuencia, tfidf. Normalize cada pesado según lo crea conveniente

In [289]:
# [alegría, enojo, miedo, repulsiun, sorpresa, tristeza]
emotions = {'alegría':0, 'enojo':1, 'miedo':2, 'repulsión':3, 'sorpresa':4, 'tristeza':5}
emolex_sel_dict = {}
with open('./SEL.txt', 'r', encoding='ISO-8859-1') as sel_file:
    next(sel_file)
    for word_enriched in sel_file:
        word_enriched_parts = word_enriched.split()
        word = word_enriched_parts[0].lower()
        emotion = word_enriched_parts[-1].lower()
        one_hot_emotion = np.zeros(6)
        one_hot_emotion[emotions[emotion]] = word_enriched_parts[1]
        emolex_sel_dict[word] = one_hot_emotion

In [290]:
print(emolex_sel_dict)

{'abundancia': array([0.83, 0.  , 0.  , 0.  , 0.  , 0.  ]), 'acabalar': array([0.396, 0.   , 0.   , 0.   , 0.   , 0.   ]), 'acallar': array([0.198, 0.   , 0.   , 0.   , 0.   , 0.   ]), 'acatar': array([0.198, 0.   , 0.   , 0.   , 0.   , 0.   ]), 'acción': array([0.397, 0.   , 0.   , 0.   , 0.   , 0.   ]), 'aceptable': array([0.594, 0.   , 0.   , 0.   , 0.   , 0.   ]), 'aceptación': array([0.696, 0.   , 0.   , 0.   , 0.   , 0.   ]), 'acicate': array([0.429, 0.   , 0.   , 0.   , 0.   , 0.   ]), 'aclamación': array([0.799, 0.   , 0.   , 0.   , 0.   , 0.   ]), 'aclamar': array([0.799, 0.   , 0.   , 0.   , 0.   , 0.   ]), 'acogedor': array([0.83, 0.  , 0.  , 0.  , 0.  , 0.  ]), 'acoger': array([0.729, 0.   , 0.   , 0.   , 0.   , 0.   ]), 'acomodadamente': array([0.729, 0.   , 0.   , 0.   , 0.   , 0.   ]), 'acuciar': array([0.264, 0.   , 0.   , 0.   , 0.   , 0.   ]), 'acucioso': array([0.232, 0.   , 0.   , 0.   , 0.   , 0.   ]), 'adecuar': array([0.331, 0.   , 0.   , 0.   , 0.   , 0.   ]), '

In [341]:
# Creamos una clase para constuir la bolsa de palabras 
from collections import Counter

class BOW_emotions_sel:
    def __init__(self, tr_text, corpus_ranking, emolex_dict, num_emotions):
        self.tr_text = tr_text
        self.corpus_ranking = corpus_ranking  
        self._N = len(tr_text)
        self._M = num_emotions # Número de emociones 
        self.bow = np.zeros((self._N, self._M))
        self.emolex_dict = emolex_dict

    def _doc_freq(self):
        """
            Función para contar el número de documentos 
            que contienen un termino 
        """
        doc_freq_dict = {}
        tweets_tokenizados = [Counter(tokenizer.tokenize(doc)) for doc in tr_text]
        for token in corpus_ranking:
            if token in doc_freq_dict:
                pass
            else:
                doc_freq_dict[token] = sum([1 for tweet_tokenizado in tweets_tokenizados if token in tweet_tokenizado])
        return doc_freq_dict

    def bow_pesado(self, pesado='binario'):
        """
            Función para rellenar la bolsa de palabras 
            de acuerdo al pesado solicitado 
        """
        if pesado == 'tf_idf':
            doc_freq_dict = self._doc_freq()
            
        for num_doc, doc in enumerate(self.tr_text):
            doc_tokenizado = Counter(tokenizer.tokenize(doc))
            for palabra, count in doc_tokenizado.items():
                if palabra in self.corpus_ranking and palabra in self.emolex_dict:
                    if pesado == 'binario':
                        self.bow[num_doc] = self.bow[num_doc] + self.emolex_dict[palabra]
                    elif pesado == 'frecuencia':
                        self.bow[num_doc] = self.bow[num_doc] + count * self.emolex_dict[palabra]
                    elif pesado == 'tf_idf':
                        self.bow[num_doc] = self.bow[num_doc] + count * np.log(self._N / doc_freq_dict[palabra]) * self.emolex_dict[palabra]
        return self.bow

    def bow_normalizado(self):
        for i in range(self._M):
            norm = np.linalg.norm(self.bow[:, i])
            if norm > 0:
                self.bow[:, i] /= norm
        return self.bow


In [298]:
# Pesado binario sin normalizar 
bow_tb_sel = BOW_emotions_sel(tr_text, corpus_ranking, emolex_sel_dict, 6)
bow_train_binario_sel = bow_tb_sel.bow

bow_vb_sel = BOW_emotions_sel(val_txt, corpus_ranking, emolex_sel_dict, 6)
bow_val_binario_sel = bow_vb_sel.bow

In [306]:
clasificacion(bow_train_binario_sel, bow_val_binario_sel, tr_y, val_y)

[[418   0]
 [169   0]]
              precision    recall  f1-score   support

           0       0.71      1.00      0.83       418
           1       0.00      0.00      0.00       169

    accuracy                           0.71       587
   macro avg       0.36      0.50      0.42       587
weighted avg       0.51      0.71      0.59       587



/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [309]:
# Pesado frecuencia normalizado 
bow_tf_sel = BOW_emotions_sel(tr_text, corpus_ranking, emolex_sel_dict, 6)
bow_tf_sel.bow_pesado(pesado='frecuencia')
bow_tf_sel.bow_normalizado()
bow_train_frecuencia_sel = bow_tf_sel.bow

bow_vf_sel = BOW_emotions_sel(val_txt, corpus_ranking, emolex_sel_dict, 6)
bow_vf_sel.bow_pesado(pesado='frecuencia')
bow_vf_sel.bow_normalizado()
bow_val_frecuencia_sel = bow_vf_sel.bow

In [310]:
clasificacion(bow_train_frecuencia_sel, bow_val_frecuencia_sel, tr_y, val_y)

[[ 76 342]
 [ 33 136]]
              precision    recall  f1-score   support

           0       0.70      0.18      0.29       418
           1       0.28      0.80      0.42       169

    accuracy                           0.36       587
   macro avg       0.49      0.49      0.35       587
weighted avg       0.58      0.36      0.33       587



In [342]:
# Pesado TF IDF normalizado 
bow_ttfidf_sel = BOW_emotions_sel(tr_text, corpus_ranking, emolex_sel_dict, 6)
bow_ttfidf_sel.bow_pesado(pesado='tf_idf')
bow_ttfidf_sel.bow_normalizado()
bow_train_tfidf_sel = bow_ttfidf_sel.bow

bow_vtfidf_sel = BOW_emotions_sel(val_txt, corpus_ranking, emolex_sel_dict, 6)
bow_vtfidf_sel.bow_pesado(pesado='tf_idf')
bow_vf_sel.bow_normalizado()
bow_val_tfidf_sel = bow_vtfidf_sel.bow

In [344]:
clasificacion(bow_train_tfidf_sel, bow_val_tfidf_sel, tr_y, val_y)

[[ 76 342]
 [ 34 135]]
              precision    recall  f1-score   support

           0       0.69      0.18      0.29       418
           1       0.28      0.80      0.42       169

    accuracy                           0.36       587
   macro avg       0.49      0.49      0.35       587
weighted avg       0.57      0.36      0.33       587



A continuación se muestran los resultados, en este caso el mejor rendimiento lo tiene el pesado TF IDF normalizado, dónde cabe resaltar que para el pesado binario, de manera sistemática predice correctamente los elementos de una clase, pero no los de la otra, por eso no se toma como el método con el mejor rendimiento.

| Método | F1-score (weighted avg) |
|:------------:|:--------------:|
| Pesado binario (sin normalizar) | 0.59 | 
| Pesado frecuencia (normalizado) | 0.33 |
| Pesado TF IDF (normalizado)     | 0.33 |

2. En un comentario aparte, discuta sobre la estrategía que utilizó para incorporar el
"Probability Factor of Affective use". No más de 5 renglones.

De manera similar a como lo hicimos con el recurso léxico del Consejo Nacional de Investigación de Canada, codificamos cada palabra en un vector sparse dónde cada entrada representa una emoción, y la entrada diferente de cero corresponde al paramétro PFA de la emoción con la que esta asociada tal palabra, después, para representar cada documento acumulamos las representaciones vectroriales de las palabras que lo contienen. 

## 4 ¿Podemos mejorar con Bigramas? (30 pts)

1. Hacer un experimento dónde concatene una buena BoW según sus experimentos anteri-
ores con otra BoW construida a partir de los 1000 bigramas más frecuentes.

In [329]:
# Primero determinamos los 1000 bigramas más frecuentes 
from nltk.util import bigrams

bigrams_list = list(bigrams(corpus))
bigrams_count = nltk.FreqDist(bigrams_list)
bigrams_sorted = [(bigrams_count[key], key) for key in bigrams_count]
bigrams_sorted.sort()
bigrams_sorted.reverse()
bigrams_sorted_filtered = bigrams_sorted[:1000]
bigrams_ranking = {}
for i, (_, key) in enumerate(bigrams_sorted_filtered):
    bigrams_ranking[key] = i
print('Ranking de los 1000 bigramas más frecuentes: ', bigrams_ranking)

# Creamos la clase para crear la bolsa concatenada 
from collections import Counter

class BOW_concatenada:
    def __init__(self, tr_text, corpus_ranking, bigramas_ranking):
        self.tr_text = tr_text
        self.corpus_ranking = corpus_ranking 
        self.bigramas_ranking = bigramas_ranking
        self._N = len(tr_text)
        self._M = len(corpus_ranking) + len(bigramas_ranking)
        self.bow = np.zeros((self._N, self._M))

    def bow_pesado(self, pesado='binario'):
        '''
            Función para construir la bolsa de palabras 
        ''' 
        for num_doc, doc in enumerate(self.tr_text):
            doc_tokenizado = Counter(tokenizer.tokenize(doc))
            doc_bigrams = Counter(list(bigrams(doc)))
            for palabra, count in doc_tokenizado.items():
                if palabra in self.corpus_ranking:
                    idx = self.corpus_ranking[palabra]
                    self.bow[num_doc, idx] = count
            for bigrama, count in doc_bigrams.items():
                if bigrama in self.bigramas_ranking:
                    idx = self.bigramas_ranking[bigrama] + len(self.corpus_ranking)
                    self.bow[num_doc, idx] = count
        return self.bow



Ranking de los 1000 bigramas más frecuentes:  {('!', '!'): 0, ('@USUARIO', '@USUARIO'): 1, ('a', 'la'): 2, ('de', 'la'): 3, ('la', 'verga'): 4, ('que', 'no'): 5, ('😂', '😂'): 6, ('que', 'me'): 7, (',', 'no'): 8, ('.', '@USUARIO'): 9, ('en', 'la'): 10, ('que', 'se'): 11, ('en', 'el'): 12, (',', 'pero'): 13, ('la', 'madre'): 14, ('lo', 'que'): 15, ('y', 'no'): 16, ('los', 'putos'): 17, ('no', 'se'): 18, ('que', 'te'): 19, ('no', 'me'): 20, ('.', 'No'): 21, ('?', '?'): 22, (',', 'que'): 23, ('puta', 'madre'): 24, ('su', 'madre'): 25, ('a', 'su'): 26, ('voy', 'a'): 27, ('para', 'que'): 28, ('las', 'putas'): 29, ('de', 'mi'): 30, (',', 'ya'): 31, ('a', 'las'): 32, ('madre', '.'): 33, ('es', 'que'): 34, ('a', 'los'): 35, ('de', 'los'): 36, ('y', 'me'): 37, ('verga', ','): 38, ('a', 'mi'): 39, ('que', 'la'): 40, (',', 'me'): 41, ('verga', '.'): 42, ('.', 'Me'): 43, ('no', 'es'): 44, ('madre', ','): 45, ('vale', 'verga'): 46, ('no', 'te'): 47, ('con', 'el'): 48, (',', 'y'): 49, ('de', 'su'): 50

In [331]:
# Construimos las bolsas concatenadas con el corpus 
bow_tc = BOW_concatenada(tr_text, corpus_ranking, bigrams_ranking)
bow_train_concatenada = bow_tc.bow

bow_vc = BOW_concatenada(val_txt, corpus_ranking, bigrams_ranking)
bow_val_concatenada = bow_vc.bow

In [334]:
# Clasificamos con SVM 
clasificacion(bow_train_concatenada, bow_val_concatenada, tr_y, val_y)

[[  0 418]
 [  0 169]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       418
           1       0.29      1.00      0.45       169

    accuracy                           0.29       587
   macro avg       0.14      0.50      0.22       587
weighted avg       0.08      0.29      0.13       587



/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

2. Hacer un experimento con las Bolsas de Emociones, Bolsa de Palabras y Bolsa de Bi-
gramas; usted elige las dimensionalidades. Para construir la representación final del documento utilice la concatenación de las representaciones según sus observaciones (e.g., Bolsa de Palabras + Bolsa de Bigramas + Bolsa de Sentimientos de Canadá + Bolsa de Sentimientos de Grigori), y aliméntelas a un SVM.

In [348]:
# Concatenemos todas las bolsas

class BOW_full:
    def __init__(self, tr_text, corpus_ranking, bigrams_ranking, emolex_dict, emolex_sel_dict):
        self.tr_text = tr_text
        self.corpus_ranking = corpus_ranking  
        self.bigrams_ranking = bigrams_ranking
        self.emolex_dict = emolex_dict
        self.emolex_sel_dict = emolex_sel_dict
        self._N = len(tr_text)
        self._M = len(corpus_ranking) + len(bigrams_ranking) + 14 # Número de emociones 
        self.bow = np.zeros((self._N, self._M))

    def bow_pesado(self, pesado='binario'):
        """
            Función para rellenar la bolsa de palabras 
            de acuerdo al pesado solicitado 
        """
            
        for num_doc, doc in enumerate(self.tr_text):
            doc_tokenizado = Counter(tokenizer.tokenize(doc)) # Tokenizamos el documento 
            doc_bigramas = Counter(list(bigrams(doc))) # Extraemooso bigramas del documento 
            for palabra, count in doc_tokenizado.items():
                if palabra in self.corpus_ranking: # Pesado binario en palabras 
                    idx = corpus_ranking[palabra]
                    self.bow[num_doc, idx] = 1
                if palabra in self.emolex_dict: # Acumulamos one-hot de emociones emolex
                    self.bow[num_doc, 5000:5008] = self.bow[num_doc, 5000:5008] + self.emolex_dict[palabra]
                if palabra in self.emolex_sel_dict: # Acumulamos one-hot de emociones de sel
                    self.bow[num_doc, 5008: 5014] = self.bow[num_doc, 5008: 5014] + self.emolex_sel_dict[palabra]
            for bigrama, count in doc_bigramas: # Pesado binario en documentos 
                if bigrama in self.bigrams_ranking:
                    idx = 5014 + bigrams_ranking[bigrama]
                    self.bow[num_doc, idx] = 1 
        return self.bow

    def bow_normalizado(self):
        for i in range(self._M):
            norm = np.linalg.norm(self.bow[:, i])
            if norm > 0:
                self.bow[:, i] /= norm
        return self.bow


In [349]:
# Creamos la bolsa con los datos 
bow_tfull = BOW_full(tr_text, corpus_ranking, bigrams_ranking, emolex_dict, emolex_sel_dict)
bow_tfull.bow_normalizado()
bow_train_full = bow_tfull.bow

bow_vfull = BOW_full(val_txt, corpus_ranking, bigrams_ranking, emolex_dict, emolex_sel_dict)
bow_vfull.bow_normalizado()
bow_val_full = bow_vfull.bow

In [350]:
# Alimentamos la bolsa de palabras, emociones y bigramas al SVM
clasificacion(bow_train_full, bow_val_full, tr_y, val_y)

[[  0 418]
 [  0 169]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       418
           1       0.29      1.00      0.45       169

    accuracy                           0.29       587
   macro avg       0.14      0.50      0.22       587
weighted avg       0.08      0.29      0.13       587



/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sergio/anaconda3/envs/machine_lr/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

3. Elabore conclusiones sobre toda esta Tarea, incluyendo observaciones, comentarios y osibles mejoras futuras. Discuta el comportamiento de la BoW de usar solo palabras
a integrar bigramas, y luego a integrar todo ¿ayudó? o ¿empeoró?. Discuta también brevemente el costo computacional de los experimentos ¿Valió la Pena tener todo?. Sea
breve: todo en NO más de dos párrafos.

Luego de llevar a cabo cada uno de los experimentos, podríamos concluir que el mejor rendimiento lo obtuvieron configuraciones sencillas como la bolsa de palabras simple con pesado binario o con pesado frecuencia. Pude observar que al integral los recurso léxicos, el rendimiento disminuia considerablemente, esto podría deberse a varios factores como el hecho de que muchas de las palabras del corpus al ser de un medio social carecen de estructura gramatical, además debido al tamaño de los recursos es posible que muchas palabras se quedaran sin enmascarar. 

Luego al incorporar los bigramas y los recurso léxicos tampoco mejoro, esto podría deberse también a que en este punto el número de características es considerablemente mayor al número total de documentos. Respecto al costo computacional, en experimentos dónde se involucraba el cálculo del TF IDF el algoritmo demoraba más, lo cuál es algo a tener en cuenta a pesar de que en general este pesado da buen rendimiento.